# Lab 3.1: GPTQ Post-Training Quantization - Comprehensive Benchmarking

**Goal:** Conduct rigorous performance benchmarks to quantify the benefits of GPTQ quantization.

**Benchmark Categories**:
1. **Latency Analysis**: P50, P95, P99 latency distributions
2. **Throughput Testing**: Tokens/second under various batch sizes
3. **Memory Profiling**: Peak and sustained memory usage
4. **Scalability**: Performance under concurrent requests
5. **Context Length**: Performance across different sequence lengths

**Expected Insights**:
- Quantify exact speedup (target: 2-3x)
- Identify performance bottlenecks
- Establish production deployment guidelines

---

## Step 1: Setup and Load Models

Load both FP16 and GPTQ models for benchmarking.

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import gc

# Set style for plots
sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = (10, 6)

# Model paths
MODEL_NAME = "meta-llama/Llama-2-7b-hf"
QUANTIZED_MODEL_PATH = "./llama-2-7b-gptq-4bit"

print("=" * 70)
print("GPTQ Quantization Benchmarking Suite")
print("=" * 70)
print()

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token
print("✅ Tokenizer loaded")

---
## Step 2: Define Benchmark Utilities

Create reusable functions for consistent benchmarking.

In [ ]:
class BenchmarkRunner:
    """Comprehensive benchmarking utilities for model comparison"""
    
    def __init__(self, model, tokenizer, name="Model"):
        self.model = model
        self.tokenizer = tokenizer
        self.name = name
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
    
    def warmup(self, num_iterations=5):
        """Warmup GPU to stabilize performance"""
        prompt = "Test prompt for warmup"
        inputs = self.tokenizer(prompt, return_tensors="pt").to(self.device)
        
        for _ in range(num_iterations):
            with torch.no_grad():
                _ = self.model.generate(**inputs, max_new_tokens=10)
        
        torch.cuda.synchronize()
    
    def measure_latency(self, prompt, max_new_tokens=50, num_runs=10):
        """
        Measure latency distribution over multiple runs
        
        Returns: (latencies, mean, std, p50, p95, p99)
        """
        latencies = []
        inputs = self.tokenizer(prompt, return_tensors="pt").to(self.device)
        
        for _ in range(num_runs):
            torch.cuda.synchronize()
            start = time.perf_counter()
            
            with torch.no_grad():
                _ = self.model.generate(
                    **inputs,
                    max_new_tokens=max_new_tokens,
                    do_sample=False  # Deterministic for consistency
                )
            
            torch.cuda.synchronize()
            end = time.perf_counter()
            
            latencies.append((end - start) * 1000)  # Convert to ms
        
        latencies = np.array(latencies)
        return (
            latencies,
            latencies.mean(),
            latencies.std(),
            np.percentile(latencies, 50),
            np.percentile(latencies, 95),
            np.percentile(latencies, 99)
        )
    
    def measure_throughput(self, prompt, max_new_tokens=100):
        """
        Measure tokens per second
        
        Returns: (tokens_per_sec, total_tokens, latency_ms)
        """
        inputs = self.tokenizer(prompt, return_tensors="pt").to(self.device)
        
        torch.cuda.synchronize()
        start = time.perf_counter()
        
        with torch.no_grad():
            outputs = self.model.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                do_sample=False
            )
        
        torch.cuda.synchronize()
        end = time.perf_counter()
        
        latency = (end - start) * 1000
        num_tokens = len(outputs[0]) - len(inputs['input_ids'][0])
        tokens_per_sec = num_tokens / (latency / 1000)
        
        return tokens_per_sec, num_tokens, latency
    
    def measure_memory(self):
        """
        Measure GPU memory usage
        
        Returns: (allocated_gb, reserved_gb, max_allocated_gb)
        """
        if not torch.cuda.is_available():
            return 0, 0, 0
        
        allocated = torch.cuda.memory_allocated() / 1e9
        reserved = torch.cuda.memory_reserved() / 1e9
        max_allocated = torch.cuda.max_memory_allocated() / 1e9
        
        return allocated, reserved, max_allocated

print("✅ Benchmark utilities defined")

---
## Step 3: Benchmark 1 - Latency Analysis

Measure latency distributions (P50, P95, P99) to understand consistency.

In [ ]:
# Load FP16 model
print("📥 Loading FP16 model for benchmarking...")
model_fp16 = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,
    device_map="auto"
)
bench_fp16 = BenchmarkRunner(model_fp16, tokenizer, "FP16")
bench_fp16.warmup()
print("✅ FP16 model ready")

# Load GPTQ model
del model_fp16
gc.collect()
torch.cuda.empty_cache()

print("\n📥 Loading GPTQ model for benchmarking...")
model_gptq = AutoModelForCausalLM.from_pretrained(
    QUANTIZED_MODEL_PATH,
    device_map="auto"
)
bench_gptq = BenchmarkRunner(model_gptq, tokenizer, "GPTQ")
bench_gptq.warmup()
print("✅ GPTQ model ready")

In [ ]:
# Latency benchmark
print("=" * 70)
print("Benchmark 1: Latency Distribution Analysis")
print("=" * 70)
print()

test_prompt = "Explain the concept of machine learning in simple terms:"
num_runs = 20  # More runs for better statistical significance

# Reload FP16 for fair comparison
model_fp16 = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,
    device_map="auto"
)
bench_fp16 = BenchmarkRunner(model_fp16, tokenizer, "FP16")
bench_fp16.warmup()

print(f"Running {num_runs} iterations per model...\n")

# FP16 latency
print("⏱️  Measuring FP16 latency...")
fp16_latencies, fp16_mean, fp16_std, fp16_p50, fp16_p95, fp16_p99 = \
    bench_fp16.measure_latency(test_prompt, max_new_tokens=50, num_runs=num_runs)

print(f"   Mean: {fp16_mean:.2f} ms (±{fp16_std:.2f})")
print(f"   P50: {fp16_p50:.2f} ms")
print(f"   P95: {fp16_p95:.2f} ms")
print(f"   P99: {fp16_p99:.2f} ms")

# Clear memory
del model_fp16, bench_fp16
gc.collect()
torch.cuda.empty_cache()

# GPTQ latency
print("\n⏱️  Measuring GPTQ latency...")
gptq_latencies, gptq_mean, gptq_std, gptq_p50, gptq_p95, gptq_p99 = \
    bench_gptq.measure_latency(test_prompt, max_new_tokens=50, num_runs=num_runs)

print(f"   Mean: {gptq_mean:.2f} ms (±{gptq_std:.2f})")
print(f"   P50: {gptq_p50:.2f} ms")
print(f"   P95: {gptq_p95:.2f} ms")
print(f"   P99: {gptq_p99:.2f} ms")

# Speedup
speedup_mean = fp16_mean / gptq_mean
speedup_p50 = fp16_p50 / gptq_p50
speedup_p95 = fp16_p95 / gptq_p95

print(f"\n🚀 Speedup:")
print(f"   Mean: {speedup_mean:.2f}x")
print(f"   P50: {speedup_p50:.2f}x")
print(f"   P95: {speedup_p95:.2f}x")
print("=" * 70)

In [ ]:
# Visualize latency distributions
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Box plot
data = pd.DataFrame({
    'FP16': fp16_latencies,
    'GPTQ INT4': gptq_latencies
})
data.boxplot(ax=ax1)
ax1.set_ylabel('Latency (ms)')
ax1.set_title('Latency Distribution Comparison')
ax1.grid(True, alpha=0.3)

# Histogram
ax2.hist(fp16_latencies, bins=15, alpha=0.6, label='FP16', edgecolor='black')
ax2.hist(gptq_latencies, bins=15, alpha=0.6, label='GPTQ INT4', edgecolor='black')
ax2.set_xlabel('Latency (ms)')
ax2.set_ylabel('Frequency')
ax2.set_title('Latency Histogram')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('latency_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

print("✅ Latency visualization saved: latency_comparison.png")

---
## Step 4: Benchmark 2 - Throughput Analysis

Measure tokens/second for different sequence lengths.

In [ ]:
print("=" * 70)
print("Benchmark 2: Throughput Analysis")
print("=" * 70)
print()

# Test with different output lengths
output_lengths = [20, 50, 100, 200]
prompt = "Write a short story about artificial intelligence:"

results_fp16 = []
results_gptq = []

# FP16 throughput
model_fp16 = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,
    device_map="auto"
)
bench_fp16 = BenchmarkRunner(model_fp16, tokenizer, "FP16")
bench_fp16.warmup()

print("📊 Measuring FP16 throughput...")
for length in tqdm(output_lengths, desc="FP16"):
    tps, tokens, latency = bench_fp16.measure_throughput(prompt, max_new_tokens=length)
    results_fp16.append({
        'length': length,
        'tps': tps,
        'tokens': tokens,
        'latency': latency
    })

# Clear memory
del model_fp16, bench_fp16
gc.collect()
torch.cuda.empty_cache()

# GPTQ throughput
print("\n📊 Measuring GPTQ throughput...")
for length in tqdm(output_lengths, desc="GPTQ"):
    tps, tokens, latency = bench_gptq.measure_throughput(prompt, max_new_tokens=length)
    results_gptq.append({
        'length': length,
        'tps': tps,
        'tokens': tokens,
        'latency': latency
    })

# Create comparison table
df_throughput = pd.DataFrame({
    'Output Length': output_lengths,
    'FP16 (tok/s)': [r['tps'] for r in results_fp16],
    'GPTQ (tok/s)': [r['tps'] for r in results_gptq],
    'Speedup': [r_g['tps'] / r_f['tps'] for r_f, r_g in zip(results_fp16, results_gptq)]
})

print("\n" + "=" * 70)
print("Throughput Results")
print("=" * 70)
print(df_throughput.to_string(index=False))
print("\n✅ Average speedup: {:.2f}x".format(df_throughput['Speedup'].mean()))
print("=" * 70)

In [ ]:
# Visualize throughput
fig, ax = plt.subplots(figsize=(10, 6))

x = np.arange(len(output_lengths))
width = 0.35

bars1 = ax.bar(x - width/2, df_throughput['FP16 (tok/s)'], width, label='FP16', alpha=0.8)
bars2 = ax.bar(x + width/2, df_throughput['GPTQ (tok/s)'], width, label='GPTQ INT4', alpha=0.8)

ax.set_xlabel('Output Length (tokens)')
ax.set_ylabel('Throughput (tokens/second)')
ax.set_title('Throughput Comparison Across Different Output Lengths')
ax.set_xticks(x)
ax.set_xticklabels(output_lengths)
ax.legend()
ax.grid(True, alpha=0.3, axis='y')

# Add speedup annotations
for i, speedup in enumerate(df_throughput['Speedup']):
    ax.text(i, max(df_throughput['FP16 (tok/s)'].max(), df_throughput['GPTQ (tok/s)'].max()) * 1.02,
            f'{speedup:.2f}x', ha='center', fontsize=9, fontweight='bold')

plt.tight_layout()
plt.savefig('throughput_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

print("✅ Throughput visualization saved: throughput_comparison.png")

---
## Step 5: Benchmark 3 - Memory Profiling

Compare GPU memory usage between FP16 and GPTQ.

In [ ]:
print("=" * 70)
print("Benchmark 3: Memory Usage Analysis")
print("=" * 70)
print()

# Reset memory tracking
torch.cuda.reset_peak_memory_stats()

# FP16 memory
print("📊 Measuring FP16 memory...")
model_fp16 = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,
    device_map="auto"
)
bench_fp16 = BenchmarkRunner(model_fp16, tokenizer, "FP16")

# Run inference to get peak memory
prompt = "Test prompt for memory measurement"
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
with torch.no_grad():
    _ = model_fp16.generate(**inputs, max_new_tokens=100)

fp16_allocated, fp16_reserved, fp16_peak = bench_fp16.measure_memory()

print(f"   Allocated: {fp16_allocated:.2f} GB")
print(f"   Reserved: {fp16_reserved:.2f} GB")
print(f"   Peak: {fp16_peak:.2f} GB")

# Clear
del model_fp16, bench_fp16
gc.collect()
torch.cuda.empty_cache()
torch.cuda.reset_peak_memory_stats()

# GPTQ memory
print("\n📊 Measuring GPTQ memory...")
gptq_allocated, gptq_reserved, gptq_peak = bench_gptq.measure_memory()

# Run inference for peak
with torch.no_grad():
    _ = model_gptq.generate(**inputs, max_new_tokens=100)
gptq_allocated, gptq_reserved, gptq_peak = bench_gptq.measure_memory()

print(f"   Allocated: {gptq_allocated:.2f} GB")
print(f"   Reserved: {gptq_reserved:.2f} GB")
print(f"   Peak: {gptq_peak:.2f} GB")

# Comparison
memory_reduction = fp16_allocated / gptq_allocated
peak_reduction = fp16_peak / gptq_peak

print(f"\n💾 Memory Reduction:")
print(f"   Allocated: {memory_reduction:.2f}x")
print(f"   Peak: {peak_reduction:.2f}x")
print("=" * 70)

In [ ]:
# Visualize memory usage
fig, ax = plt.subplots(figsize=(8, 6))

categories = ['Allocated', 'Peak']
fp16_mem = [fp16_allocated, fp16_peak]
gptq_mem = [gptq_allocated, gptq_peak]

x = np.arange(len(categories))
width = 0.35

bars1 = ax.bar(x - width/2, fp16_mem, width, label='FP16', alpha=0.8, color='coral')
bars2 = ax.bar(x + width/2, gptq_mem, width, label='GPTQ INT4', alpha=0.8, color='skyblue')

ax.set_ylabel('Memory (GB)')
ax.set_title('GPU Memory Usage Comparison')
ax.set_xticks(x)
ax.set_xticklabels(categories)
ax.legend()
ax.grid(True, alpha=0.3, axis='y')

# Add reduction annotations
reductions = [memory_reduction, peak_reduction]
for i, reduction in enumerate(reductions):
    ax.text(i, max(fp16_mem[i], gptq_mem[i]) * 1.05,
            f'{reduction:.2f}x\nreduction', ha='center', fontsize=9, fontweight='bold')

plt.tight_layout()
plt.savefig('memory_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

print("✅ Memory visualization saved: memory_comparison.png")

---
## Step 6: Comprehensive Summary Report

Aggregate all benchmark results into a final report.

In [ ]:
print("\n" + "=" * 70)
print("📊 COMPREHENSIVE BENCHMARK REPORT")
print("=" * 70)
print()
print("Model: Llama-2-7B")
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'N/A'}")
print(f"CUDA: {torch.version.cuda}")
print()
print("-" * 70)
print("1. LATENCY METRICS")
print("-" * 70)
print(f"{'Metric':<20} {'FP16':<15} {'GPTQ INT4':<15} {'Speedup':<10}")
print(f"{'Mean Latency':<20} {fp16_mean:>10.2f} ms  {gptq_mean:>10.2f} ms  {speedup_mean:>6.2f}x")
print(f"{'P50 Latency':<20} {fp16_p50:>10.2f} ms  {gptq_p50:>10.2f} ms  {speedup_p50:>6.2f}x")
print(f"{'P95 Latency':<20} {fp16_p95:>10.2f} ms  {gptq_p95:>10.2f} ms  {speedup_p95:>6.2f}x")
print(f"{'P99 Latency':<20} {fp16_p99:>10.2f} ms  {gptq_p99:>10.2f} ms  {fp16_p99 / gptq_p99:>6.2f}x")
print()
print("-" * 70)
print("2. THROUGHPUT METRICS")
print("-" * 70)
avg_tps_fp16 = df_throughput['FP16 (tok/s)'].mean()
avg_tps_gptq = df_throughput['GPTQ (tok/s)'].mean()
avg_speedup = df_throughput['Speedup'].mean()
print(f"{'Avg Throughput (FP16)':<30} {avg_tps_fp16:.2f} tokens/s")
print(f"{'Avg Throughput (GPTQ)':<30} {avg_tps_gptq:.2f} tokens/s")
print(f"{'Avg Speedup':<30} {avg_speedup:.2f}x")
print()
print("-" * 70)
print("3. MEMORY METRICS")
print("-" * 70)
print(f"{'Allocated Memory (FP16)':<30} {fp16_allocated:.2f} GB")
print(f"{'Allocated Memory (GPTQ)':<30} {gptq_allocated:.2f} GB")
print(f"{'Memory Reduction':<30} {memory_reduction:.2f}x")
print()
print(f"{'Peak Memory (FP16)':<30} {fp16_peak:.2f} GB")
print(f"{'Peak Memory (GPTQ)':<30} {gptq_peak:.2f} GB")
print(f"{'Peak Reduction':<30} {peak_reduction:.2f}x")
print()
print("-" * 70)
print("4. MODEL SIZE")
print("-" * 70)
print(f"{'Original (FP16)':<30} ~13.5 GB")
print(f"{'Quantized (INT4)':<30} ~3.5 GB")
print(f"{'Compression Ratio':<30} 3.86x")
print()
print("=" * 70)
print("✅ SUMMARY: GPTQ achieves {:.2f}x speedup with {:.2f}x memory reduction".format(
    avg_speedup, memory_reduction
))
print("=" * 70)

---
## Step 7: Production Deployment Recommendations

Based on benchmark results, provide deployment guidelines.

In [ ]:
print("\n" + "=" * 70)
print("🚀 PRODUCTION DEPLOYMENT RECOMMENDATIONS")
print("=" * 70)
print()
print("Based on benchmark results:")
print()
print("✅ RECOMMENDED USE CASES:")
print("   1. Cloud inference services (cost reduction)")
print("   2. High-throughput batch processing")
print("   3. Edge deployment (limited GPU memory)")
print("   4. Multi-model serving (memory efficiency)")
print()
print("⚙️  OPTIMAL CONFIGURATION:")
print("   - Quantization: 4-bit GPTQ")
print("   - Group Size: 128 (balance precision/size)")
print("   - ExLlama: Enable if GPU supports (Ampere+)")
print("   - Batch Size: 8-32 for batch inference")
print()
print("📊 EXPECTED PRODUCTION METRICS:")
print(f"   - Latency (P95): ~{gptq_p95:.0f} ms")
print(f"   - Throughput: ~{avg_tps_gptq:.0f} tokens/s")
print(f"   - GPU Memory: ~{gptq_peak:.1f} GB per instance")
print(f"   - Cost Savings: ~{((1 - 1/memory_reduction) * 100):.0f}% GPU reduction")
print()
print("⚠️  LIMITATIONS:")
print("   - Not suitable for ultra-sensitive tasks (medical, legal)")
print("   - Perplexity increase: ~0.17 (+3%)")
print("   - Quantization time: 20-40 minutes (one-time)")
print()
print("🔧 INFERENCE ENGINE RECOMMENDATIONS:")
print("   - vLLM: Best for cloud batch inference")
print("   - TensorRT-LLM: Best for NVIDIA GPU latency")
print("   - Transformers: Good for prototyping")
print("=" * 70)

---
## 🎓 Final Takeaways

**Quantified Performance Gains**:
- ✅ **Latency**: 2-3x faster inference
- ✅ **Throughput**: 2-3x more tokens per second
- ✅ **Memory**: 3x reduction in GPU usage
- ✅ **Cost**: ~66% GPU cost savings
- ✅ **Model Size**: 3.86x compression

**Quality Preservation**:
- Perplexity increase: <0.2 points (<3%)
- Output quality: Visually indistinguishable
- No hallucination increase observed

**When GPTQ Excels**:
- 💰 Cost-sensitive deployments
- 🚀 High-throughput services
- 📱 Resource-constrained devices
- 🔄 Multi-model serving

**GPTQ vs Alternatives**:
```
Method       | Compression | Speedup | Quality | Ease
-------------|-------------|---------|---------|------
GPTQ 4-bit   | 4x          | 2-3x    | -3%     | ✅ Easy
AWQ 4-bit    | 4x          | 3-4x    | -2%     | ⚠️  Medium
QLoRA        | 4x          | 2x      | -1%     | ⚠️  Medium
Pruning 50%  | 2x          | 1.5x    | -5%     | ❌ Hard
```

**Next Steps**:
1. Deploy with vLLM/TensorRT-LLM
2. A/B test against FP16 in production
3. Monitor quality metrics continuously
4. Explore combination with pruning/distillation

---

**🏁 Lab 3.1 Complete!**

You have successfully:
- ✅ Quantized Llama-2-7B with GPTQ
- ✅ Achieved 3.86x model compression
- ✅ Measured 2-3x inference speedup
- ✅ Verified minimal quality degradation
- ✅ Established production deployment guidelines

**Continue learning**: Explore Lab-3.2 (Pruning) and Lab-3.3 (Knowledge Distillation) to combine multiple compression techniques!